In [ ]:
import xml.etree.ElementTree as ET
import string

def extract_labels(sentence, aspect_terms):
    words = sentence.strip().split()
    labels = ['O'] * len(words)

    for aspect_term in aspect_terms:
        aspect_words = aspect_term.split()
        if aspect_words and aspect_words[0] in words:
            start_index = words.index(aspect_words[0])
            labels[start_index] = 'B'
            for i in range(1, len(aspect_words)):
                if start_index + i < len(words) and words[start_index + i] == aspect_words[i]:
                    labels[start_index + i] = 'I'

    return labels

def read_xml_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

def remove_punctuation(sentence):
    translator = str.maketrans('', '', string.punctuation)
    return sentence.translate(translator)

if __name__ == "__main__":
    xml_file_path = "/content/temple/temple_3.xml"
    root = read_xml_file(xml_file_path)
    sentences=[]
    aspect_termss=[]
    labelss=[]
    # Print sentences, aspect terms, and labels
    for sentence_node in root.findall('.//sentence'):
        sentence = remove_punctuation(sentence_node.text.strip().lower())
        aspect_terms = [aspect_node.attrib['term'].lower() for aspect_node in sentence_node.findall('.//aspectTerm')]
        labels = extract_labels(sentence, aspect_terms)
        sentences.append(sentence)
        aspect_termss.append(aspect_terms)
        labelss.append(labels)
        print("Sentence:", sentence)
        print("Aspect Terms:", aspect_terms)
        print("Labels:", labels)
        print()


In [ ]:
import pandas as pd

df=pd.DataFrame()
df['Sentences']=sentences
df['aspect terms']=aspect_termss
df["Labels"]=labelss

In [ ]:
df.to_csv('temple_3.csv')

In [ ]:
import pandas as pd

# Load the existing CSV files
csv1 = pd.read_csv("/content/temple_1.csv")
csv2 = pd.read_csv("/content/temple_2.csv")
csv3 = pd.read_csv("/content/temple_3.csv")


appended_csv = pd.concat([csv1, csv2,csv3], ignore_index=True)


# Save the appended CSV to a new file
appended_csv.to_csv("temple_final.csv", index=False)


In [ ]:
import pandas as pd

templet_df=pd.read_excel('/content/temple_final.xlsx')
temple_df

,Sentences,aspect terms,Labels
0,this famous temple of goddess bhagavathi at ch...,['religious shrine'],"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,thronged by devotees this temple is known for ...,['cures'],"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,the temple has a historical legacy,['historical legacy'],"['O', 'O', 'O', 'O', 'B', 'I']"
3,the goddess here is worshipped in three forms,['goddess'],"['O', 'B', 'O', 'O', 'O', 'O', 'O', 'O']"
4,temple has facilities attached for stay of the...,['facilities attached for stay'],"['O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'O']"
...,...,...,...
858,the local kings made huge donations in gold g...,['gold'],"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', ..."
859,hence its one of the richest temples richer th...,['temples'],"['O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', ..."
860,the temple is only open for hindus,['temple'],"['O', 'B', 'O', 'O', 'O', 'O', 'O']"
861,non hindus are allowed to see temple from outside,['outside'],"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']"


In [ ]:
def check_aspect_term_length(aspect_terms, labels):
    if set(aspect_terms)=={'null'}:
      return True
    aspect_words_count = sum(len(aspect_term.split()) for aspect_term in aspect_terms)
    b_and_i_count = sum(1 for label in labels if label == 'B' or label == 'I')
    if 'null' in set(aspect_terms):
      return aspect_words_count-aspect_terms.count('null') == b_and_i_count
    elif len(set(aspect_terms))<len(aspect_terms):
      return aspect_words_count-(len(aspect_terms)-len(set(aspect_terms))) == b_and_i_count
    else:
      return aspect_words_count == b_and_i_count

# Example usage:
for i in range(len(temple_df['aspect terms'])):
  aspect_terms=eval(temple_df['aspect terms'][i])
  labels = eval(temple_df['Labels'][i])
  result = check_aspect_term_length(aspect_terms, labels)
  if result==False:
    print(i+2,result)
